# Reading, Writing and Validating Data in PySpark HW

Welcome to your first coding homework assignment in PySpark! I hope you enjoyed the lecture on Reading, Writing and Validating dataframes. Now it's time to put what you've learned into action! 

I've included several instructions below to help guide you through this homework assignment which I hope will get you feeling even comfortable reading, writing and validating dataframes. If you get stuck at any point, feel free to jump to the next lecture where I will guide you through my solutions to the HW assignment. 

Have fun!

Let's dig right in!


## But first things first.....
We need to always begin every Spark session by creating a Spark instance. Let's go ahead and use the method we learned in the lecture in the cell below. Also see if you can remember how to open the Spark UI (using a link that automatically guides you there). 

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [ ]:
# Install pyspark
!pip install pyspark
# import findspark
# findspark.init()
import os
import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession
# May take awhile locally
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

cores = spark._jsc.sc().getExecutorMemoryStatus().keySet().size()
print("You are working with", cores, "core(s)")
spark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
You are working with 1 core(s)


In [ ]:
basedir = '/content/drive/MyDrive/Data Science Intake43/5. Spark/spark-scripts/section2/Datasets'
os.listdir(basedir)

['people.json',
 'zomato.csv',
 'youtubevideos.csv',
 'googleplaystore.csv',
 'fifa19.csv',
 'students.csv',
 'users1.parquet',
 'users3.parquet',
 'users2.parquet',
 'nyc_air_bnb.csv',
 'rec-crime-pfa.csv',
 'supermarket_sales.csv',
 'Rep_vs_Dem_tweets.csv',
 'pga_tour_historical.csv',
 'uw-madison-courses']

## Next let's start by reading a basic csv dataset

Download the pga_tour_historical dataset that is attached to this lecture and save it whatever folder you want, then read it in. 

**Data Source:** https://www.kaggle.com/bradklassen/pga-tour-20102018-data

Rememer to try letting Spark infer the header and infer the Schema types!

In [ ]:
path = os.path.join(basedir,'pga_tour_historical.csv')

In [ ]:
# Start by reading a basic csv dataset
# Let Spark know about the header and infer the Schema types!

df = spark.read.csv(path,header=True,inferSchema=True)

## 1. View first 5 lines of dataframe
First generate a view of the first 5 lines of the dataframe to get an idea of what is inside. We went over two ways of doing this... see if you can remember BOTH ways. 

In [ ]:
df.show(5)

+---------------+------+----------------+--------------------+-----+
|    Player Name|Season|       Statistic|            Variable|Value|
+---------------+------+----------------+--------------------+-----+
|Robert Garrigus|  2010|Driving Distance|Driving Distance ...|   71|
|   Bubba Watson|  2010|Driving Distance|Driving Distance ...|   77|
| Dustin Johnson|  2010|Driving Distance|Driving Distance ...|   83|
|Brett Wetterich|  2010|Driving Distance|Driving Distance ...|   54|
|    J.B. Holmes|  2010|Driving Distance|Driving Distance ...|  100|
+---------------+------+----------------+--------------------+-----+
only showing top 5 rows



In [ ]:
df.limit(5).toPandas()

,Player Name,Season,Statistic,Variable,Value
0,Robert Garrigus,2010,Driving Distance,Driving Distance - (ROUNDS),71
1,Bubba Watson,2010,Driving Distance,Driving Distance - (ROUNDS),77
2,Dustin Johnson,2010,Driving Distance,Driving Distance - (ROUNDS),83
3,Brett Wetterich,2010,Driving Distance,Driving Distance - (ROUNDS),54
4,J.B. Holmes,2010,Driving Distance,Driving Distance - (ROUNDS),100


## 2. Print the schema details

Now print the details of the dataframes schema that Spark infered to ensure that it was infered correctly. Sometimes it is not infered correctly, so we need to watch out!

In [ ]:
df.printSchema()

root
 |-- Player Name: string (nullable = true)
 |-- Season: integer (nullable = true)
 |-- Statistic: string (nullable = true)
 |-- Variable: string (nullable = true)
 |-- Value: string (nullable = true)



## 3. Edit the schema during the read in

We can see from the output above that Spark did not correctly infer that the "value" column was an integer value. Let's try specifying the schema this time to let spark know what the schema should be.

Here is a link to see a list of PySpark data types in case you need it (also attached to the lecture): 
https://spark.apache.org/docs/latest/sql-ref-datatypes.html

In [ ]:
from pyspark.sql.types import StructField,StringType,IntegerType,StructType,DateType
data_schema = [StructField("Player Name", StringType(), True),
               StructField("Season", IntegerType(), True),
               StructField("Statistics", StringType(), True),
               StructField("variable", StringType(), True),
               StructField("Value", IntegerType(), True)]
final_struc = StructType(fields=data_schema)
df = spark.read.csv(path,header=True, schema = final_struc)

In [ ]:
df.printSchema()

root
 |-- Player Name: string (nullable = true)
 |-- Season: integer (nullable = true)
 |-- Statistics: string (nullable = true)
 |-- variable: string (nullable = true)
 |-- Value: integer (nullable = true)



## 4. Generate summary statistics for only one variable

See if you can generate summary statistics for only the "Value" column using the .describe function

(count, mean, stddev, min, max) 

In [ ]:
# Neat "describe" function
df.describe(['Value']).show()

+-------+------------------+
|summary|             Value|
+-------+------------------+
|  count|           1657247|
|   mean|12494.388998743096|
| stddev|157274.75673570752|
|    min|              -178|
|    max|           3564954|
+-------+------------------+



## 5. Generate summary statistics for TWO variables
Now try to generate ONLY the count min and max for BOTH the "Value" and "Season" variable using the select. You can't use the .describe function for this one but see if you can remember which function you CAN use. 

In [ ]:
df.select('Value', 'Season').summary("count", "min", "max").show()

+-------+-------+-------+
|summary|  Value| Season|
+-------+-------+-------+
|  count|1657247|2740403|
|    min|   -178|   2010|
|    max|3564954|   2018|
+-------+-------+-------+



In [ ]:
df.describe(['Value', 'Season']).show()

+-------+------------------+-----------------+
|summary|             Value|           Season|
+-------+------------------+-----------------+
|  count|           1657247|          2740403|
|   mean|12494.388998743096|2013.973479083186|
| stddev|157274.75673570752|2.607050115512023|
|    min|              -178|             2010|
|    max|           3564954|             2018|
+-------+------------------+-----------------+



## 6. Write a parquet file

Now try writing a parquet file (not partitioned) from the pga dataset. But first create a new dataframe containing ONLY the the "Season" and "Value" fields (using the "select command you used in the question above) and write a parquet file partitioned by "Season". This is a bit of a challenge aimed at getting you ready for material that will be covered later on in the course. Don't feel bad if you can't figure it out.

*Note that if any of your variable names contain spaces, spark will produce an error message with this call. That is why we are selecting ONLY the "Season" and "Value" fields. Ideally we should renamed those columns but we haven't gotten to that yet in this course but we will soon!*

In [ ]:
df_prq = df.select('Value', 'Season')
df_prq.show()

+-----+------+
|Value|Season|
+-----+------+
|   71|  2010|
|   77|  2010|
|   83|  2010|
|   54|  2010|
|  100|  2010|
|   63|  2010|
|   88|  2010|
|   64|  2010|
|   64|  2010|
|   92|  2010|
|   75|  2010|
|   54|  2010|
|   76|  2010|
|   94|  2010|
|   82|  2010|
|   85|  2010|
|   79|  2010|
|   89|  2010|
|   88|  2010|
|   91|  2010|
+-----+------+
only showing top 20 rows



In [ ]:
df_prq.write.mode("overwrite").parquet('/content/drive/MyDrive/Data Science Intake43/5. Spark/Labs/Fatma Gad/parquet/')

## 7. Write a partioned parquet file

You will need to use the same limited dataframe that you created in the previous question to accomplish this task as well. 

In [ ]:
df_prq.write.mode("overwrite").partitionBy("Season").parquet('/content/drive/MyDrive/Data Science Intake43/5. Spark/Labs/Fatma Gad/part_parquet/')

## 8. Read in a partitioned parquet file

Now try reading in the partitioned parquet file you just created above. 

In [ ]:
prq_path = '/content/drive/MyDrive/Data Science Intake43/5. Spark/Labs/Fatma Gad/part_parquet/'

In [ ]:
partitioned = spark.read.parquet(prq_path)
# partitioned = spark.read.parquet(prq_path+'Season*')
partitioned.show(5)

+-----+------+
|Value|Season|
+-----+------+
|   71|  2010|
|   77|  2010|
|   83|  2010|
|   54|  2010|
|  100|  2010|
+-----+------+
only showing top 5 rows



## 9. Reading in a set of paritioned parquet files

Now try only reading Seasons 2010, 2011 and 2012.

In [ ]:
first_3_seasons = spark.read.option("basePath", prq_path).parquet(prq_path+'Season=2010', prq_path+'Season=2011', prq_path+'Season=2012')
first_3_seasons.show(5)

+-----+------+
|Value|Season|
+-----+------+
|   71|  2010|
|   77|  2010|
|   83|  2010|
|   54|  2010|
|  100|  2010|
+-----+------+
only showing top 5 rows



In [ ]:
first_3_seasons.select('Season').distinct().collect()

[Row(Season=2012), Row(Season=2010), Row(Season=2011)]

## 10. Create your own dataframe

Try creating your own dataframe below using PySparks *.createDataFrame* function. See if you can make one that contains 4 variables and at least 3 rows. 

Let's see how creative you can get on the content of the dataframe :)

In [ ]:
students_data = [('Fatma Gad',12, 3.4),('Aliaa Adel',11, 3.7),
                 ('Salma Amr',12, 3.85),('Eshraq Saeed',9, 3.6),
                 ('Amira Amr',9,3.55),('Salma Eissa',5, 3.7), 
                 ('Aya Medhat',8, 3.6) ]
students = spark.createDataFrame(students_data,['Full_Name', 'Grade', 'GPA'])
students.show()

+------------+-----+----+
|   Full_Name|Grade| GPA|
+------------+-----+----+
|   Fatma Gad|   12| 3.4|
|  Aliaa Adel|   11| 3.7|
|   Salma Amr|   12|3.85|
|Eshraq Saeed|    9| 3.6|
|   Amira Amr|    9|3.55|
| Salma Eissa|    5| 3.7|
|  Aya Medhat|    8| 3.6|
+------------+-----+----+



In [ ]:
students.toPandas()

,Full_Name,Grade,GPA
0,Fatma Gad,12,3.40
1,Aliaa Adel,11,3.70
2,Salma Amr,12,3.85
3,Eshraq Saeed,9,3.60
4,Amira Amr,9,3.55
5,Salma Eissa,5,3.70
6,Aya Medhat,8,3.60


## We're done! Great job!